In [ ]:
%load_ext autoreload
%autoreload 2
from nbodysim.Simulator import Simulator
from nbodysim.Analyzer import Analyzer
from nbodysim.Utils import periapsisFinder,rotate3DVector
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pos,vel=periapsisFinder(7.2e31, 1.2e11, eccentricity=0, inclination=0)
initialmomentum=(5.8e31)*(vel)
dV=(-(initialmomentum)/(7.2e31+5.8e31))

dP=-(5.8e31)*(pos)/(7.2e31+5.8e31)

print(pos,dP)
print(dV)

In [ ]:
sim2 = Simulator(name='Hyperstar')

sim2.addObject(name='BH1',mass=7.2e31,radius=1.9e10,position=dP,velocity=dV,color=(0,0,255))

sim2.addObject(name='BH2',mass=5.8e31,radius=6.2e9,position=pos+dP,velocity=vel+dV,color=(255,0,0))


#mag = 1e5
#direction = (np.pi)/(10)
#HVSvelocity = np.array([(mag)*np.cos(direction),(mag)*np.sin(direction),0])
pos1,vel1 = periapsisFinder(7.2e31,1e12,.95,0)
pos1 += dP
pos1 = rotate3DVector(pos1,'z',np.pi,False)
vel1 = rotate3DVector(vel1,'z',np.pi,False)
sim2.addObject(name='HVS' ,mass=2e30, radius=7e5, position=pos1, velocity=vel1, color=(0,255,0))


In [ ]:
print(sim2.getTotalEnergy('HVS'))
v = (sim2.getObject('HVS').velocity)
magnitude = np.sqrt(np.sum(np.square(v)))
print(magnitude)
print(vel1)


In [ ]:
sim2.plot(axes=('x','y'),plotRange=None,plotSize=(600,600),trails=True)

In [ ]:
sim2.play(dt=10,numberOfSteps=60*6,save=True,plotFirst=False)

In [ ]:
ana = Analyzer('Hyperstar')
ana.plot(['BH1','BH2','HVS'],['vy','vy','vy'])

In [ ]:
time = ana.objectData['HVS'][:,0]
vel2 = ana.objectData['HVS'][:,6:9]

m = np.zeros(len(time))

for i in range(len(time)):
    c = vel2[i]
    m[i] = np.sqrt(c.dot(c))



In [ ]:
plt.figure(figsize = (10,8))
plt.title("Magnitude of Velocity of HVS")
plt.xlabel('time in seconds')
plt.ylabel('Magnitude of Velocity in m/s')
plt.plot(time,m)
plt.show()

In [ ]:
z = self.getObject(nameOrIndex)
kineticEnergy = (1/2)*(z.mass)*(z.velocity.dot(z.velocity))
        
potentialEnergy = 0
        
for i in range(self.objectCount):
            
    y = self.allObjects[i]
            
    if y != z:
        dif = ((z.position[0]-y.position[0]))
        pE = (-self.G*z.mass*y.mass)/np.sqrt(dif.dot(dif))
        potentialEnergy += pE 
                
return(kineticEnergy + potentialEnergy)

In [ ]:
time = ana.objectData['HVS'][:,0]
vel2 = ana.objectData['HVS'][:,6:9]
targetMass = ana.objectData['HVS'][0,1]
targetPosition = ana.objectData['HVS'][:,3:6]
G = 6.67e-11

e = np.zeros(len(time))
k = np.zeros(len(time))
p = np.zeros(len(time))

for t in range(len(time)):
    currentVel = vel2[t]
    ke = currentVel.dot(currentVel)*targetMass*(1/2)
    currentPosition = targetPosition[t]
    pe = 0
    

    for key in (ana.objectData):
        if key != 'HVS':
            otherPosition = ana.objectData[key][t,3:6]
            otherMass = ana.objectData[key][0,1]
            dif = ((currentPosition - otherPosition))
            pe += (-G*targetMass*otherMass)/np.sqrt(dif.dot(dif))
            
    e[t] = ke + pe
    k[t] = ke
    p[t] = pe
    


In [ ]:
plt.figure(figsize = (10,8))
plt.title("Total Energy of HVS")
plt.xlabel('time in seconds')
plt.ylabel('Total Energy in Joules')
plt.plot(time,e,label='Total Energy')
plt.plot(time,k,label='Kinetic Energy')
plt.plot(time,p,label='Potential Energy')
plt.legend(loc='upper left')
plt.show()